In [4]:
#Standard Lib imports
import sys, os
from pathlib import Path
import getpass
from tqdm import tqdm_notebook
from time import sleep


# Third party imports
import pandas as pd

# Local application imports
from sei import sei
from selenium.webdriver.support.ui import Select

#USR = getpass.getuser()
USR = 'rsilva'
print(f'Olá {USR}')
PWD = getpass.getpass("Digite sua Senha: ")
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'pandas'

In [2]:
!pip list

Package            Version  
------------------ ---------
appdirs            1.4.3    
aspy.yaml          1.3.0    
atomicwrites       1.3.0    
attrs              19.3.0   
backcall           0.1.0    
beautifulsoup4     4.8.1    
black              19.10b0  
cfgv               2.0.1    
Click              7.0      
colorama           0.4.3    
coverage           4.5.4    
decorator          4.4.1    
entrypoints        0.3      
flake8             3.7.9    
identify           1.4.8    
importlib-metadata 1.3.0    
ipykernel          5.1.3    
ipython            7.10.1   
ipython-genutils   0.2.0    
isort              4.3.21   
jedi               0.15.1   
jupyter-client     5.3.4    
jupyter-core       4.6.1    
mccabe             0.6.1    
more-itertools     8.0.2    
mypy               0.750    
mypy-extensions    0.4.3    
nodeenv            1.3.3    
numpy              1.17.4   
packaging          19.2     
pandas             0.25.3   
parso              0.5.1    
pathspec      

In [5]:
??unicodedata

In [ ]:
s = sei.login_sei(USR, PWD, browser='Firefox', timeout=1, teste=True)

In [3]:
df = pd.read_excel(FILE, dtype=str)

In [7]:
df['Situação do Processo'].unique()

array(['Concluído', 'Aberto'], dtype=object)

In [26]:
df['Alterado'] = "Não"

In [45]:
def alterar_acesso(link, nivel='Restrito', hip='Informação Pessoal (Art. 31 da Lei nº 12.527/2011)'):    

    s.go(link)
    p = sei.Processo(s.page, row["Processo"])
    sleep(1)
    
    acoes = p._get_acoes(click=False)
#     if acoes.get("Reabrir Processo"):
#         with p._go_to_central_frame():
#             p.page._click_button_new_win(acoes['Reabrir Processo'])
#             return
            
#         s.go(link)
#         p = sei.Processo(s.page, row["Processo"])
#         sleep(1)
    
    acao = acoes.get("Consultar/Alterar Documento")
    if not acao:
        acao = acoes.get("Consultar/Alterar Documento Externo")
        
    if acao:
        with p._go_to_central_frame():
            p.page._click_button_new_win(acao)
            p.page._clicar(('xpath', f'//*[@id="opt{nivel}"]'))                
            if nivel == 'Restrito':
                hip = Select(p.page.wait_for_element_to_click(('xpath', '//*[@id="selHipoteseLegal"]')))
                hip.select_by_value("35") #.select_by_visible_text(hip)                    
            p.page._clicar(('xpath', '//*[@id="btnSalvar"]'))
            return True
    
    return False

In [26]:
sub = df[df['Situação do Processo'] == "Concluir"]
for i, row in tqdm_notebook(sub.iterrows(), total=sub.shape[0]):
    p = s.go(row['Link SEI'])
    p = sei.Processo(s.page, row["Processo"])
    #sleep(1)
    #p.concluir_processo()
    df.loc[i, "Situação do Processo"] = "OK"

In [49]:
sub = df[df.Alterado == "Erro"]
for i, row in tqdm_notebook(sub.iterrows(), total=sub.shape[0]):
    if alterar_acesso(link=row['Link SEI'], nivel='Restrito', hip=r'Fiscalização / Investigação da Anatel (Art. 174 da Lei nº 9.472/1997)'):
        df.loc[i, 'Alterado'] = 'Sim'
    else:
        df.loc[i, 'Alterado'] = 'Erro'    

In [51]:
df.to_csv(FILE.parent / 'FI3.csv', index=False)